In [3]:
import os

import dotenv

# Get the path and load the .env file
dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)

True

In [5]:
import os
from openai import OpenAI

YOUR_API_KEY = os.getenv("PERPLEXITY_API_KEY")

# Initialize the client
client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

def get_ai_response(prompt):
    messages = [
        {
            "role": "system",
            "content": "You are an artificial intelligence assistant, and you need to respond in a helpful, detailed, polite conversation with a user."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    try:
        response = client.chat.completions.create(
            model="llama-3.1-sonar-large-128k-online",  # You can adjust the model as needed
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error occurred: {str(e)}"

def main():
    # Example usage
    prompt = "What does Cotswold Industries provide?"
    response = get_ai_response(prompt)
    print(response)

if __name__ == "__main__":
    main()

Cotswold Industries, Inc. is a comprehensive manufacturer, converter, and distributor of a wide range of textiles and fabrics, with a diverse portfolio of products and services. Here are some key areas where they provide their offerings:

## Technical Textiles and Engineered Fabrics
Cotswold Industries is a leader in the innovation and distribution of technical textiles and engineered fabrics. They produce a variety of products for different markets, including:
- **Military and Government**: They supply fabrics to correctional industries programs, federal prison systems, and other government entities. These include woven and knitted fabrics such as broadcloth, sheeting, poplin, pocketing, denim, and various knit fabrics like jersey and fleece.
- **Medical Markets**: They offer medical fabrics, personal protective equipment, cleanroom fabrics, and other specialized medical textiles.
- **Commercial Applications**: Their products are used in commercial wallcovering, vinyl coating, and oth

In [9]:
import asyncio
from typing import List, Dict
from openai import OpenAI
import logging
import nest_asyncio
import sys

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

class FinanceQAApp:
    def __init__(self, api_key: str):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://api.perplexity.ai"
        )
        self.model = "llama-3.1-sonar-large-128k-online"
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def generate_related_questions(self, main_question: str) -> List[str]:
        """Generate related questions using GPT."""
        messages = [
            {
                "role": "system",
                "content": (
                    "You are a financial analysis assistant. Generate up to 5 specific, "
                    "related questions that would help provide a comprehensive answer to "
                    "the main question. Return only the questions, one per line, "
                    "without numbering or additional text."
                )
            },
            {
                "role": "user",
                "content": main_question
            }
        ]

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            questions = response.choices[0].message.content.strip().split('\n')
            return questions[:5]  # Ensure we return maximum 5 questions
        except Exception as e:
            self.logger.error(f"Error generating related questions: {e}")
            return []

    async def search_perplexity(self, question: str) -> Dict:
        """Perform a search query using Perplexity API."""
        messages = [
            {
                "role": "system",
                "content": "You are a financial research assistant. Provide clear, concise answers with relevant facts and figures."
            },
            {
                "role": "user",
                "content": question
            }
        ]

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            return {
                "question": question,
                "answer": response.choices[0].message.content
            }
        except Exception as e:
            self.logger.error(f"Error searching Perplexity for '{question}': {e}")
            return {
                "question": question,
                "answer": f"Error: Unable to retrieve answer - {str(e)}"
            }

    async def process_question(self, main_question: str) -> Dict:
        """Process the main question and return comprehensive results."""
        # Generate related questions
        related_questions = self.generate_related_questions(main_question)
        
        # Create tasks for parallel processing
        tasks = [self.search_perplexity(question) for question in [main_question] + related_questions]
        
        # Execute searches in parallel
        results = await asyncio.gather(*tasks)
        
        return {
            "main_question": main_question,
            "main_answer": results[0]["answer"],
            "related_results": results[1:] if len(results) > 1 else []
        }

async def run_async_task(coro):
    """Helper function to run async code in different environments."""
    try:
        if sys.platform == 'win32':
            asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        
        return await coro
    except Exception as e:
        print(f"Error running async code: {e}")
        return None

def main():
    # Example usage
    api_key= os.getenv("PERPLEXITY_API_KEY")
    app = FinanceQAApp(api_key)
    
    # Example financial question
    question = "What are the potential impacts of rising interest rates on the stock market?"
    
    try:
        # Get the current event loop or create a new one
        try:
            loop = asyncio.get_event_loop()
        except RuntimeError:
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
        
        # Run the async task and get results
        results = loop.run_until_complete(run_async_task(app.process_question(question)))
        
        if results:
            # Print results
            print(f"\nMain Question: {results['main_question']}")
            print(f"\nMain Answer: {results['main_answer']}")
            print("\nRelated Questions and Answers:")
            for result in results['related_results']:
                print(f"\nQ: {result['question']}")
                print(f"A: {result['answer']}")
        else:
            print("Error processing the question")
            
    except Exception as e:
        print(f"Error in main: {e}")
        
if __name__ == "__main__":
    main()

INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
/opt/anaconda3/envs/woodshed/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


KeyboardInterrupt: 

INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"


: 

In [7]:
from typing import List, Dict
from openai import OpenAI
import logging
from concurrent.futures import ThreadPoolExecutor
from threading import Lock

class FinanceQAApp:
    def __init__(self, api_key: str):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://api.perplexity.ai"
        )
        self.model = "llama-3.1-sonar-large-128k-online"
        self.api_lock = Lock()  # Add thread-safe lock for API calls
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def generate_related_questions(self, main_question: str) -> List[str]:
        """Generate related questions using GPT."""
        messages = [
            {
                "role": "system",
                "content": (
                    "You are a financial analysis assistant. Generate up to 5 specific, "
                    "related questions that would help provide a comprehensive answer to "
                    "the main question. Return only the questions, one per line, "
                    "without numbering or additional text."
                )
            },
            {
                "role": "user",
                "content": main_question
            }
        ]

        try:
            with self.api_lock:  # Ensure thread-safe API calls
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages
                )
            questions = response.choices[0].message.content.strip().split('\n')
            return questions[:5]  # Ensure we return maximum 5 questions
        except Exception as e:
            self.logger.error(f"Error generating related questions: {e}")
            return []

    def search_perplexity(self, question: str) -> Dict:
        """Perform a search query using Perplexity API."""
        messages = [
            {
                "role": "system",
                "content": "You are a financial research assistant. Provide clear, concise answers with relevant facts and figures."
            },
            {
                "role": "user",
                "content": question
            }
        ]

        try:
            with self.api_lock:  # Ensure thread-safe API calls
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages
                )
            return {
                "question": question,
                "answer": response.choices[0].message.content
            }
        except Exception as e:
            self.logger.error(f"Error searching Perplexity for '{question}': {e}")
            return {
                "question": question,
                "answer": f"Error: Unable to retrieve answer - {str(e)}"
            }

    def process_question(self, main_question: str, max_workers: int = 5) -> Dict:
        """Process the main question and return comprehensive results."""
        # Generate related questions
        related_questions = self.generate_related_questions(main_question)
        all_questions = [main_question] + related_questions
        results = []

        # Use ThreadPoolExecutor for parallel processing
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all questions for processing
            future_to_question = {
                executor.submit(self.search_perplexity, question): question 
                for question in all_questions
            }

            # Collect results as they complete
            for future in future_to_question:
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    self.logger.error(f"Error processing question: {e}")

        # Separate main question result from related questions
        main_result = next(r for r in results if r["question"] == main_question)
        related_results = [r for r in results if r["question"] != main_question]

        return {
            "main_question": main_question,
            "main_answer": main_result["answer"],
            "related_results": related_results
        }

def main():
    # Example usage
    api_key= os.getenv("PERPLEXITY_API_KEY")
    app = FinanceQAApp(api_key)
    
    # Example financial question
    question = "What are the potential impacts of rising interest rates on the stock market?"
    
    # Process the question
    results = app.process_question(question)
    
    # Print results
    print(f"\nMain Question: {results['main_question']}")
    print(f"\nMain Answer: {results['main_answer']}")
    print("\nRelated Questions and Answers:")
    for result in results['related_results']:
        print(f"\nQ: {result['question']}")
        print(f"A: {result['answer']}")

if __name__ == "__main__":
    main()

INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"



Main Question: What are the potential impacts of rising interest rates on the stock market?

Main Answer: The potential impacts of rising interest rates on the stock market are multifaceted and can be summarized as follows:

## Increased Borrowing Costs
Rising interest rates increase the cost of borrowing for companies, which can eat into their profits and make their stocks less attractive to investors. This higher cost of borrowing can slow down expansion plans, reduce profit margins, and ultimately lead to lower stock prices.

## Sector Rotation
Different sectors of the stock market react differently to rising interest rates. Financial stocks, such as banks and insurance companies, tend to perform well because they can charge higher interest rates on loans, boosting their profit margins. In contrast, sectors like utilities, which rely heavily on borrowing, and technology stocks with high growth potential, may suffer as their dividend yields become less attractive compared to rising 